In [118]:
import pandas as pd
import numpy as np
import random
from random import *

In [3]:
dict_nuc = {1:"A", 2:"T", 3:"C", 4:"G"}

In [5]:
def make_pat(num_patterns, len_pattern):
    set_pats = set()
    dict_nuc = {1:"A", 2:"T", 3:"C", 4:"G"}
    
    while len(set_pats) != num_patterns:
        s_temp = ''.join(pd.Series(np.random.randint(low=1, high=4, size=len_pattern)).map(dict_nuc).values)
        set_pats.add(s_temp)
    return set_pats
        

In [6]:
make_pat(5,3)

{'AAT', 'CCA', 'CTC', 'TAA', 'TAT'}

In [105]:
def add_pats_to_str(pat, string, gap = 1):
    locs = np.random.choice(np.arange(0,len(string) - len(pat[0]), len(pat[0])+gap ), size=len(pat), replace=False)
    locs = sorted(locs)
    print(locs)
    for i in range(len(locs)):
        string = string[:locs[i]] + pat[i] + string[locs[i]+len(pat[i]):]
    return string, locs

In [104]:
add_pats_to_str(["AAT", "TAC", "ATG"], "00000000000000000")

[0, 8, 12]


('AAT00000TAC0ATG00', [0, 8, 12])

In [78]:
def make_str_without_pats(pat, len_string):
    string = ""
    dict_nuc = {1:"A", 2:"T", 3:"C", 4:"G"}
    s_temp = ''.join(pd.Series(np.random.randint(low=1, high=4, size=len_string)).map(dict_nuc).values)
    locs = [s_temp.find(x) for x in pat]
    locs = list(filter((-1).__ne__, locs)).copy()
    kkk = 2
    while len(locs) != 0:
        for idx in locs:
            list_nuc = ["A","T", "C", "G"]
            list_nuc.remove(s_temp[idx])
            s_temp = s_temp[:idx] + np.random.choice(list_nuc, size=1)[0] + s_temp[idx+1:] 
        locs = [s_temp.find(x) for x in pat]
        locs = list(filter((-1).__ne__, locs)).copy()
        kkk -=1
    return s_temp


In [106]:
make_str_without_pats(["AAT", "ACT", "TTT"], 40)

'CACATATCTCCCAAAAAACCGATATATACCACACATTCTC'

In [107]:
df = pd.DataFrame(columns = ["String", "Patterns", "Location", "Exclusion"])

num_strings = 5
len_strings = 20
len_pattern = 3
num_patterns= 3
pats = make_pat(num_patterns, len_pattern) # make patterns list
pats = list(pats)
while num_strings > 0:
#     randomly choose the number of patters to be in this string
#     np.random.randint(low=1, high=4, size=len_pattern)
    st = make_str_without_pats(pats, len_strings) # create strings without pattern list
    st, locs = add_pats_to_str(pats, st) # add patern list to string at random places with atleast 1 gap
    new_row = {"String" : st, "Patterns": pats, "Location": locs}
    df = df.append(new_row, ignore_index=True)
    num_strings -=1

        
df  

[0, 8, 16]
[0, 4, 12]
[0, 8, 12]
[0, 8, 16]
[0, 8, 16]


/var/folders/_8/tqnhz5dn2q77dh1l3q_btskc0000gn/T/ipykernel_2510/2308054454.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/var/folders/_8/tqnhz5dn2q77dh1l3q_btskc0000gn/T/ipykernel_2510/2308054454.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/var/folders/_8/tqnhz5dn2q77dh1l3q_btskc0000gn/T/ipykernel_2510/2308054454.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/var/folders/_8/tqnhz5dn2q77dh1l3q_btskc0000gn/T/ipykernel_2510/2308054454.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df 

,String,Patterns,Location,Exclusion
0,TTCACCTCACAATCACACTA,"[TTC, ACA, ACT]","[0, 8, 16]",NaN
1,TTCAACACATAAACTCAGCT,"[TTC, ACA, ACT]","[0, 4, 12]",NaN
2,TTCCTAAAACACACTTCATT,"[TTC, ACA, ACT]","[0, 8, 12]",NaN
3,TTCCACCAACAAATCTACTC,"[TTC, ACA, ACT]","[0, 8, 16]",NaN
4,TTCCCAGCACACCATGACTC,"[TTC, ACA, ACT]","[0, 8, 16]",NaN


In [109]:
df

,String,Patterns,Location,Exclusion
0,TTCACCTCACAATCACACTA,"[TTC, ACA, ACT]","[0, 8, 16]",NaN
1,TTCAACACATAAACTCAGCT,"[TTC, ACA, ACT]","[0, 4, 12]",NaN
2,TTCCTAAAACACACTTCATT,"[TTC, ACA, ACT]","[0, 8, 12]",NaN
3,TTCCACCAACAAATCTACTC,"[TTC, ACA, ACT]","[0, 8, 16]",NaN
4,TTCCCAGCACACCATGACTC,"[TTC, ACA, ACT]","[0, 8, 16]",NaN


20

In [127]:
# %%time
best_motifs = []
min_score = 10000
count = 0
while count < 1000:
    motifs = GibbsSampler(df["String"].values, len_pattern, len(df), min(df["String"].str.len()) )
    ss = Score(motifs)
    if ss < min_score:
        best_motifs = motifs
        min_score = ss
        print(ss)
    count += 1

print(best_motifs)

6
5
3
2
0
['ACA', 'ACA', 'ACA', 'ACA', 'ACA']


In [128]:
# # questions
# 1. How approx is the sol
# 2. Explain exclusion string -> still not clear
# 3. Order vs approx sol

In [125]:
def ProfileProb(seq, k, prob_dic):
    minn = 0
    min_pat = []
    for i in range(len(seq) - k + 1):
        pat = seq[i:i+k]
        seq_sum = 0
        for j in range(k):
            seq_sum += prob_dic[pat[j]][j]
        if minn < seq_sum:
            minn = seq_sum
            min_pat = pat
    return min_pat

def Score(motiff):
    k = len(motiff[0])
    score_mat = {"A": [0]*k, "C": [0]*k, "G": [0]*k, "T": [0]*k}
    for nei in motiff:
        idx = 0
        for nuc in str(nei):
            score_mat[nuc][idx] +=1
            idx+=1
    score = 0
    smat = list(zip(*score_mat.values()))
    for i in range(k):
            score = score + sum(smat[i]) - max(smat[i])
    return score

def Profile_creator(motiff):
    k = len(motiff[0])
    profile_matrix = {"A": [1]*k, "C": [1]*k, "G": [1]*k, "T": [1]*k}
    for nei in motiff:
        idx = 0
        for nuc in str(nei):
            profile_matrix[nuc][idx] +=1
            idx+=1
    for j in range(k):
        col_sum =0
        for i in ["A", "C", "G", "T"]:
            col_sum += profile_matrix[i][j]
        for i in ["A", "C", "G", "T"]:
            profile_matrix[i][j] = profile_matrix[i][j]/col_sum
    return profile_matrix

def RandomMotif(seq, k, prob_dic):
    rand_dic = {}    
    for i in range(len(seq) - k +1):
        pat = seq[i:i+k]
        seq_sum = 1
        for j in range(k):
            seq_sum *= prob_dic[pat[j]][j]
        rand_dic[pat] = seq_sum
        
    keey = choices(list(rand_dic.keys()), weights=list(rand_dic.values()))  
    return keey[0]

def GibbsSampler(dna_list, k, t, N):
    motiff = []
    for seq in dna_list:
        rand = randint(0, len(seq)- k)
        motiff.append(seq[rand: rand + k])
    best_motif = motiff
    for j in range(N):
        i = randint(0,t-1)
#         print(best_motif)
#         print(i)
        prof = Profile_creator(best_motif[:i] + best_motif[i+1:])
        ith_motif = RandomMotif(dna_list[i], k, prof)
#         print(ith_motif)
        motiff = best_motif[:i] + [ith_motif] + best_motif[i+1:]
        if Score(motiff) < Score(best_motif):
            best_motif = motiff.copy()
        return best_motif